https://www.kaggle.com/datasets/santoshd3/bank-customers/data?select=Churn+Modeling.csv

# 1. Импорт библиотек

In [1]:
# Импорт библиотек
import pandas as pd  # Работа с данными
import numpy as np  # Числовые операции
import matplotlib.pyplot as plt  # Визуализация
import seaborn as sns  # Красивые графики
from sklearn.preprocessing import StandardScaler  # Нормализация данных
from sklearn.cluster import KMeans  # Кластеризация
from sklearn.decomposition import PCA  # Для визуализации кластеров

# 2. Загрузка и просмотр данных

In [2]:
# Загрузка данных (замени путь на свой, если файл локальный)
data = pd.read_csv('bank_clients_data/bank_clients_data.csv')

# Просмотр первых 5 строк
display(data.head())

# Основная информация о датасете
print(data.info())

# Проверка на пропущенные значения
print(data.isnull().sum())

FileNotFoundError: [Errno 2] No such file or directory: 'bank_clients_data/bank_clients_data.csv'

На основе данных можно сделать следующие выводы:

1. **Объем данных**:  
   Датсет содержит 10 000 записей (строк) и 14 столбцов. Это достаточно большой объем данных для анализа.


2. **Отсутствие пропущенных значений**:  
   В каждом столбце отсутствуют пропущенные значения (все столбцы имеют 10 000 непустых записей). Это упрощает предварительную обработку данных, так как не требуется заполнение или удаление пропусков.


3. **Типы данных**:  
   - **Числовые данные**: Большинство столбцов содержат числовые данные (`int64` и `float64`), такие как `CreditScore`, `Age`, `Balance`, `EstimatedSalary` и другие.  
   - **Категориальные данные**: Есть три столбца с типом `object`, которые, скорее всего, содержат категориальные данные: `Surname`, `Geography`, и `Gender`. Эти данные могут потребовать преобразования в числовой формат (например, с помощью one-hot encoding) для использования в моделях машинного обучения.


4. **Особенности данных**:  
   - **CreditScore**: Кредитный рейтинг клиентов.  
   - **Geography**: Клиенты представляют разные географические регионы (например, Франция, Испания).  
   - **Gender**: В данных присутствуют клиенты обоих полов.  
   - **Age**: Возраст клиентов.  
   - **Balance**: Баланс на счетах клиентов.  
   - **NumOfProducts**: Количество продуктов, используемых клиентами.  
   - **HasCrCard**: Наличие кредитной карты.  
   - **IsActiveMember**: Активность клиента.  
   - **EstimatedSalary**: Оценка зарплаты клиентов.
   - **Exited**: показатель ухода клиента из банка.  


Данные выглядят чистыми и готовыми для дальнейшего анализа и построения моделей машинного обучения.

# 3. Предобработка данных

In [ ]:
# Удаляем ненужные столбцы (они не влияют на рекомендации)
data.drop(['RowNumber', 'Surname'], axis=1, inplace=True)

# Кодируем категориальные переменные (заменяем текст на числа)
data = pd.get_dummies(data, columns=['Geography', 'Gender'], drop_first=True)

# Нормализуем числовые признаки (чтобы KMeans работал корректно)
scaler = StandardScaler()
scaled_features = scaler.fit_transform(data)

# Преобразуем обратно в DataFrame
data_scaled = pd.DataFrame(scaled_features, columns=data.columns)

In [ ]:
display(data_scaled) # выводим таблицу с нормализированными данными

# 4. Поиск оптимального количества кластеров

In [ ]:
# Определяем количество кластеров с помощью метода локтя
inertia = []
K = range(1, 15) # Проверим от 1 до 15 кластеров

for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(data_scaled)
    inertia.append(kmeans.inertia_) # Записываем ошибку (inertia)

# Визуализация метода локтя
plt.figure(figsize=(8, 5))
plt.plot(K, inertia)
plt.xlabel('Количество кластеров')
plt.ylabel('Inertia (ошибка)')
plt.title('Метод локтя для выбора количества кластеров')
plt.show()

На графике видно, что после 4-го кластера ошибка уменьшается медленнее. Дальнейшее исследование будем проводить для 4-х кластеров.

# 5. Кластеризация клиентов (KMeans)

In [ ]:
# Обучаем KMeans с 4 кластерами
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
data_scaled['Cluster'] = kmeans.fit_predict(data_scaled) # Добавляем номер кластера в данные

# Смотрим, сколько клиентов в каждом кластере
print(data_scaled['Cluster'].value_counts())

**Кластеризация выполнена успешно**: алгоритм KMeans разделил данные на 5 кластеров.  
**Распределение клиентов по кластерам** относительно равномерное, без сильного перекоса.  


**Интерпретация**:  
   - Кластеры могут представлять различные группы клиентов с похожими характеристиками (например, по возрасту, балансу, кредитному рейтингу и т.д.).  
   - Для дальнейшего анализа можно исследовать средние значения признаков в каждом кластере, чтобы понять, чем отличаются группы клиентов.

# 6. Визуализация кластеров (PCA)

In [ ]:
# Снижаем размерность до 2D для визуализации
pca = PCA(n_components=2)
data_scaled['PCA1'] = pca.fit_transform(data_scaled)[:, 0]
data_scaled['PCA2'] = pca.fit_transform(data_scaled)[:, 1]

# Рисуем график кластеров
plt.figure(figsize=(10, 6))
sns.scatterplot(data=data_scaled, x='PCA1', y='PCA2', hue='Cluster', palette='viridis')
plt.title('Визуализация кластеров клиентов')
plt.show()

# 7. Оценка качества кластеризации
Проверим, насколько хорошо KMeans разбил данные на кластеры. Используем:
- Silhouette Score – показывает, насколько точки внутри кластера похожи друг на друга.
- Davies-Bouldin Index – оценивает компактность и разделимость кластеров.

In [ ]:
from sklearn.metrics import silhouette_score, davies_bouldin_score

# Вычисляем коэффициент силуэта
silhouette = silhouette_score(data_scaled.drop(columns=['Cluster']), data_scaled['Cluster'])
print(f'Silhouette Score (KMeans): {silhouette:.4f}')

# Вычисляем индекс Дэвиса-Болдина
db_index = davies_bouldin_score(data_scaled.drop(columns=['Cluster']), data_scaled['Cluster'])
print(f'Davies-Bouldin Index (KMeans): {db_index:.4f}')

Интерпретация метрик:

- Silhouette Score: чем ближе к 1 – тем лучше (обычно 0.5+ хорошо).
- Davies-Bouldin Index: чем меньше – тем лучше (оптимально <1).

Silhouette Score 0.2343 – это довольно низкое значение, значит, точки внутри кластеров не очень похожи друг на друга.  
Davies-Bouldin Index 1.5220 – тоже не идеальный (чем меньше, тем лучше), то есть кластеры не очень разделены.

Это говорит о том, что KMeans не идеально работает с этим датасетом – возможно, кластеры не сферические, и KMeans их размывает.  
Попробуем кластеризацию с **DBSCAN**.

# 8. Кластеризация с DBSCAN
- DBSCAN хорош для данных, где кластеры имеют нечеткие границы.
- Автоматически игнорирует выбросы, не относя их ни к одному кластеру.

Параметры:
- eps — максимальное расстояние между точками в одном кластере.
- min_samples — минимальное количество точек, чтобы создать кластер.

In [ ]:
from sklearn.neighbors import NearestNeighbors

# Подбираем значение eps с использованием K-дистанс
neighbors = NearestNeighbors(n_neighbors=15)  # Используем min_samples = 15
neighbors_fit = neighbors.fit(data_scaled.drop(columns=['Cluster', 'DBSCAN_Cluster']))
distances, indices = neighbors_fit.kneighbors(data_scaled.drop(columns=['Cluster', 'DBSCAN_Cluster']))

# Сортируем расстояния для построения графика
distances = np.sort(distances[:, -1], axis=0)

# Строим график K-дистанс
plt.figure(figsize=(8, 6))
plt.plot(distances)
plt.title('График K-дистанции для подбора eps')
plt.xlabel('Точки')
plt.ylabel('Расстояние до 4-й ближайшей точки')
plt.show()

**Интерпретация графика:**  
Выбор eps: ищем “плато” на графике, где расстояния резко возрастают. Это и будет хорошее значение для eps. На текущем графике эта точка равна 3.

In [ ]:
from sklearn.cluster import DBSCAN

# Запускаем DBSCAN (eps расчитали с K-дистанс, min_samples подобрали)
dbscan = DBSCAN(eps=3, min_samples=15)
data_scaled['DBSCAN_Cluster'] = dbscan.fit_predict(data_scaled.drop(columns=['Cluster']))

# Проверяем, сколько кластеров нашел DBSCAN
print(data_scaled['DBSCAN_Cluster'].value_counts())

# Визуализируем кластеры DBSCAN
plt.figure(figsize=(10, 6))
sns.scatterplot(data=data_scaled, x='PCA1', y='PCA2', hue='DBSCAN_Cluster', palette='viridis')
plt.title('Кластеры DBSCAN')
plt.show()

In [ ]:
# Проверяем Silhouette Score и Davies-Bouldin Index для DBSCAN
if len(set(data_scaled['DBSCAN_Cluster'])) > 1:
    silhouette_dbscan = silhouette_score(data_scaled.drop(columns=['Cluster', 'DBSCAN_Cluster']), data_scaled['DBSCAN_Cluster'])
    print(f'Silhouette Score (DBSCAN): {silhouette_dbscan:.4f}')
    
    db_index_dbscan = davies_bouldin_score(data_scaled.drop(columns=['Cluster', 'DBSCAN_Cluster']), data_scaled['DBSCAN_Cluster'])
    print(f'Davies-Bouldin Index (DBSCAN): {db_index_dbscan:.4f}')

Silhouette Score и Davies-Bouldin Index показывают, что DBSCAN плохо справляется с кластеризацией. Попробуем **GMM**.

# 9. Кластеризация с GMM

GMM — это метод кластеризации, который предполагает, что данные могут быть сгенерированы с помощью смеси нескольких нормальных распределений. Это позволяет моделировать данные, которые могут не следовать простой форме, такой как круги или эллипсы, как в KMeans.

In [1]:
# Импортируем нужную библиотеку
from sklearn.mixture import GaussianMixture

In [2]:
# Настроим и обучим модель GMM
# n_components - количество кластеров
gmm = GaussianMixture(n_components=4, random_state=42)

# Обучаем модель на данных, исключая уже имеющиеся столбцы с кластеризацией
gmm.fit(data_scaled.drop(columns=['Cluster', 'DBSCAN_Cluster']))

# Получаем метки кластеров
data_scaled['GMM_Cluster'] = gmm.predict(data_scaled.drop(columns=['Cluster', 'DBSCAN_Cluster']))

# Посмотрим на количество записей в каждом кластере
print(data_scaled['GMM_Cluster'].value_counts())

NameError: name 'data_scaled' is not defined